In [5]:
!pip install -q transformers accelerate sentencepiece
from transformers import pipeline

qa = pipeline("text2text-generation", model="google/flan-t5-small", device_map="auto")

SAFETY_NOTE = (
"I am not a doctor. This is general information only and not medical advice. "
"For diagnosis, treatment, or emergencies, consult a licensed clinician or call local emergency services."
)

SYSTEM_PROMPT = (
"Act as a friendly, careful health information assistant. "
"Keep answers short and clear. Avoid giving medical diagnosis or prescribing medication. "
"Include a brief safety note."
)

def safe_answer(user_query: str, max_new_tokens=128):
    # Simple safety filter
    risky = any(k in user_query.lower() for k in [
        "suicide", "kill myself", "overdose", "stop my meds", "dose for", "prescribe"
    ])
    if risky:
        return ("I’m really sorry you’re feeling this way. Please contact local emergency services or a suicide prevention hotline immediately. "
                "You are not alone and help is available right now.")

    prompt = f"{SYSTEM_PROMPT}\n\nQuestion: {user_query}\nAnswer:"
    out = qa(prompt, max_new_tokens=max_new_tokens, temperature=0.2)[0]["generated_text"]
    return f"{out}\n\n{SAFETY_NOTE}"

examples = [
"What causes a sore throat?",
"Is paracetamol safe for children?"
]
for q in examples:
    print("Q:", q)
    print("A:", safe_answer(q))
    print("-"*60)

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Q: What causes a sore throat?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


A: A sore throat.

I am not a doctor. This is general information only and not medical advice. For diagnosis, treatment, or emergencies, consult a licensed clinician or call local emergency services.
------------------------------------------------------------
Q: Is paracetamol safe for children?
A: No, paracetamol is not safe for children.

I am not a doctor. This is general information only and not medical advice. For diagnosis, treatment, or emergencies, consult a licensed clinician or call local emergency services.
------------------------------------------------------------
